In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import json
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dropout, Flatten, Dense
from tensorflow.python.keras import optimizers
# specifically for cnn
from tensorflow.python.keras.layers import Dropout, Flatten,Activation
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, BatchNormalization


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
# pro_dir =  "C:/Users/Landy/Desktop/Angela/prodetection2/"
# train_dir = "C:/Users/Landy/Desktop/Angela/prodetection2/train/train/"
# test_dir = "C:/Users/Landy/Desktop/Angela/prodetection2/test/test"
# train_df = pd.read_csv("C:/Users/Landy/Desktop/Angela/prodetection2/train.csv")
# train_df = train_df.sort_values(by='category',ascending=True)
# train_df.head()
pro_dir =  "C:/Users/User/Desktop/prodetection/"
train_dir = "C:/Users/User/Desktop/prodetection/train/train"
test_dir = "C:/Users/User/Desktop/prodetection/test/test"
train_df = pd.read_csv("C:/Users/User/Desktop/prodetection/train.csv")
train_df = train_df.sort_values(by='category',ascending=True)
train_df.head()

,filename,category
55585,9f89d9582e4d811330e83e3628b84114.jpg,0
53785,0e506b549c296564b55ed42283b87f64.jpg,0
53784,096f4541edcd6d8c209512f4a1a149a9.jpg,0
53783,89e70f1b22897e15782cda74fae117fc.jpg,0
53782,7f4c6b4f2177fa92d0d94b4ee2003ed3.jpg,0


In [3]:
# IMG_SIZE = 80
IMG_SIZE = 256


In [5]:
#2.0.0
df = train_df.groupby('category')['filename'].nunique()
#Series
df

category
0     2683
1     2702
2     2687
3     2703
4     2703
5     2641
6     2641
7     2660
8     2700
9     2698
10    2672
11    1843
12    2691
13    2682
14    2684
15    2632
16    2665
17    1553
18    2103
19    2679
20    2653
21    2598
22    2623
23    2540
24    2705
25    2692
26    2684
27    2702
28    2561
29    2138
30    2705
31    2677
32    2157
33     573
34    2599
35    2658
36    2686
37    1725
38    2673
39    2678
40    2681
41    2662
Name: filename, dtype: int64

In [34]:
#隨機各抽573+存成dataframe
# size = 573   # sample size
# replace = True  # with replacement
# fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
# regular_df=train_df.groupby('category', as_index=False).apply(fn)

In [4]:
# regular_df = regular_df.groupby('category')['filename'].nunique()
#Series
regular_df=pd.read_csv(pro_dir+'regular_df')
regular_df.reset_index(drop=True,inplace=True)  
# regular_df.to_csv('regular_df',index=False)

from sklearn.utils import shuffle
regular_df=shuffle(regular_df)
regular_df['category']=regular_df['category'].astype('str')
regular_df

,filename,category
8513,341b0262184add60b809aff3aed81833.jpg,14
8153,54e53adaaa9008a0a8882d7bda6dfe71.jpg,14
642,ee555dbf81daaba395cfa2a6fbf11b82.jpg,1
6781,d1816b72812038c113ba15e8f0a8fb2a.jpg,11
2153,6ee1b33a162dbe24d6786329c1f64733.jpg,3
...,...,...
20492,9dbe96caabf70bb1a4239f771a01ae96.jpg,35
5215,289cca155a2c8e87e85c386f8dc5d575.jpg,9
21408,fd2ec6ec670490d2b31b3b0e6746cbcc.jpg,37
19054,9455d8cf6f38f906124e3c9384bda82c.jpg,33


In [18]:
from sklearn.model_selection import train_test_split
train_df,valid_df=train_test_split(regular_df,test_size=0.2, shuffle = True, random_state=666, stratify=regular_df['category'])

In [32]:
train_df111= train_df.groupby('category')['filename'].nunique()
train_df111

category
0     423
1     427
10    419
11    408
12    416
13    415
14    414
15    418
16    425
17    395
18    408
19    409
2     420
20    426
21    420
22    410
23    431
24    437
25    414
26    417
27    424
28    431
29    413
3     423
30    424
31    428
32    410
33    312
34    435
35    412
36    424
37    409
38    421
39    423
4     424
40    423
41    418
5     412
6     426
7     413
8     416
9     430
Name: filename, dtype: int64

In [5]:
filenamer=regular_df['filename'].tolist() 
path='C:\\Users\\User\\Desktop\\prodetection\\train_sample\\'
# for filename in os.listdir(path):
#     if filename not in filenamer:
#         full_file_path = os.path.join(path, filename)
#         os.remove(full_file_path)

In [ ]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40, # 角度值，0~180，影象旋轉
    shear_range=0.2, # 隨機錯切換角度
    validation_split = 0.13,
    fill_mode='nearest' # 填充新建立畫素的方法

)
# # 1 2 4 7 8 14 16 28 56 112 941 1882 3764 6587 7528 13174 15056 26348 52696 
# train_generator = datagen.flow_from_dataframe(
#     dataframe=regular_df,
#     directory = path,
#     target_size=(IMG_SIZE,IMG_SIZE),
#     subset="training",
#     # batch_size=int(len(train_df)/56),
#     x_col='filename',
#     y_col='category',
#     batch_size=7110,
#     shuffle=True,
#     class_mode="categorical"
# )
# #generator will return batch_x，batch_y
# X, y = train_generator.next()

# 1 2 4 7 8 14 16 28 56 112 941 1882 3764 6587 7528 13174 15056 26348 52696 
train_df_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory = path,
    target_size=(IMG_SIZE,IMG_SIZE),
    subset="training",
    # batch_size=int(len(train_df)/56),
    x_col='filename',
    y_col='category',
    batch_size=5689,
    shuffle=True,
    class_mode="categorical"
)
#generator will return batch_x，batch_y
X_train, y_train = train_df_generator.next()

Found 5689 validated image filenames belonging to 42 classes.


In [ ]:
# 1 2 4 7 8 14 16 28 56 112 941 1882 3764 6587 7528 13174 15056 26348 52696 
valid_generator = datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory = path,
    target_size=(IMG_SIZE,IMG_SIZE),
    subset="validation",
    # batch_size=int(len(train_df)/56),
    x_col='filename',
    y_col='category',
    batch_size=212,
    seed=666,
    shuffle=True,
    class_mode="categorical"
)
#generator will return batch_x，batch_y
X_valid, y_valid = valid_df_generator.next()

In [8]:
# from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle = True, random_state=666, stratify=y )

In [6]:
import h5py
data_name = "convert_img(%s x %s)" % (IMG_SIZE,IMG_SIZE)
labels_name = "labels(%s x %s)" % (IMG_SIZE,IMG_SIZE)

In [10]:
# save converted data as hdf5
with h5py.File( pro_dir + data_name + '52696'+'.h5', 'w') as hf:
    hf.create_dataset(data_name,  data=X )
with h5py.File( pro_dir + labels_name + '52696'+'.h5', 'w') as hf:
    hf.create_dataset(labels_name,  data=y)
print("Successfully saved.")

Successfully saved.


In [11]:
# X = []
# y = []
#read/write,file must exist
# load model2
# model2 = h5py.File( pro_dir +"model2/model2"+ '.h5', 'r')
hfX = h5py.File( pro_dir + data_name +'.h5', 'r')
hfy = h5py.File( pro_dir + labels_name +'.h5', 'r')
    
X = np.array(hfX[data_name][:]) #dataset_name is same as hdf5 object name 
y = np.array(hfy[labels_name][:])

In [ ]:
X_train.shape

In [30]:
# np.set_printoptions(threshold=np.inf)
X_valid.shape

(7110, 256, 256, 3)

In [22]:
# from tensorflow.keras.applications import EfficientNetB3
from efficientnet.tfkeras import EfficientNetB3

# include_top=False(是否包含頂部(Top) 『完全連接層』)
conv_base = EfficientNetB3(weights='imagenet',include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3)) 

In [23]:
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras.layers import LeakyReLU
from tensorflow.python.keras import regularizers

# model = models.Sequential()
# model.add(conv_base)
# model.add(layers.GlobalMaxPooling2D(name="gap"))
# # model.add(layers.Flatten(name="flatten"))
# # model.add(BatchNormalization())
# # model.add(layers.Dense(256, activation='relu', name="fc1"))
# model.add(layers.Dense(42, activation='softmax', name="fc_out"))
# for layer in conv_base.layers:
#     layer.trainable = False
model = models.Sequential()
model.add(conv_base)
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1024,kernel_regularizer=regularizers.l2(0.01)))
model.add(LeakyReLU(alpha=0.1))
model.add(layers.Dropout(0.5))
# model.add(layers.Dense(256, activation='relu', name="fc1"))
model.add(layers.Dense(42, activation='softmax', name="fc_out"))
for layer in conv_base.layers:
    layer.trainable = False

In [63]:
for layer in conv_base.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = True

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b3 (Model)      (None, 8, 8, 1536)        10783528  
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 1536)        0         
_________________________________________________________________
dense_1 (Dense)              (None, 8, 8, 1024)        1573888   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 8, 8, 1024)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 8, 8, 1024)        0         
_________________________________________________________________
fc_out (Dense)               (None, 8, 8, 42)          43050     
Total params: 12,400,466
Trainable params: 1,616,938
Non-trainable params: 10,783,528
__________________________________

In [66]:
opt = tf.keras.optimizers.RMSprop(learning_rate=1e-3)

model.compile(optimizer = opt ,loss='categorical_crossentropy',metrics=['categorical_accuracy'])


In [12]:
# batch_size = int(len(train_df)/256) 1
fitbatch_size = 1
nb_epoch = 3
# STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size

In [13]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
EARLY_STOP_PATIENCE = 3

cb_early_stopper = EarlyStopping(monitor = 'categorical_accuracy', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = pro_dir+'model5/model5_best.h5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [52]:
from tensorflow.python.keras.models import save_model
from tensorflow.python.keras.models import load_model

# model.save(pro_dir+'model4/model4_batch1882and1.h5')
model=load_model(pro_dir+'model5/model5_best.h5')
print('Model Loaded.')

Model Loaded.


In [28]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4,decay=1e-6)

model.compile(optimizer = opt ,loss='categorical_crossentropy',metrics=['categorical_accuracy'])
# Train model
history = model.fit(
              X_train,y_train,
              batch_size = 1,
              validation_data=(X_valid,y_valid),
              epochs = 10,
              callbacks=[cb_checkpointer, cb_early_stopper],
#               validation_steps=3128/batch_size,
              verbose = 1)

ValueError: A target array with shape (7110, 42) was passed for an output of shape (None, 8, 8, 42) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.

In [26]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
test_df = pd.read_csv(pro_dir+'test.csv')
# convert test to array
test_datagen = ImageDataGenerator(
    rescale=1./255
)
# 1 2 3 6 9 18 677 1354 2031 4062 6093
test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    directory = test_dir,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=1,
    x_col = 'filename',
    y_col = None,
    shuffle=False,
    class_mode=None,
    seed = 666
)
#generator will return batch_x，batch_y
test_img  = test_generator.next()

Found 12186 validated image filenames.


In [39]:
# X[100:103]

array([[[[0.8470589 , 0.86274517, 0.86666673],
         [0.8470589 , 0.86274517, 0.86666673],
         [0.8470589 , 0.86274517, 0.86666673],
         ...,
         [0.8000001 , 0.8000001 , 0.8078432 ],
         [0.8000001 , 0.8000001 , 0.8078432 ],
         [0.7843138 , 0.7843138 , 0.79215693]],

        [[0.8431373 , 0.8588236 , 0.86274517],
         [0.8470589 , 0.86274517, 0.86666673],
         [0.854902  , 0.8705883 , 0.8745099 ],
         ...,
         [0.7960785 , 0.7960785 , 0.80392164],
         [0.79215693, 0.79215693, 0.8000001 ],
         [0.7803922 , 0.7803922 , 0.78823537]],

        [[0.854902  , 0.8705883 , 0.8745099 ],
         [0.854902  , 0.8705883 , 0.8745099 ],
         [0.854902  , 0.8705883 , 0.8745099 ],
         ...,
         [0.79215693, 0.7843138 , 0.7960785 ],
         [0.79215693, 0.7843138 , 0.78823537],
         [0.7803922 , 0.77647066, 0.7607844 ]],

        ...,

        [[0.8235295 , 0.83921576, 0.8431373 ],
         [0.8235295 , 0.83921576, 0.8431373 ]

In [ ]:
test_name = "test_img(%s x %s)" % (IMG_SIZE,IMG_SIZE)

with h5py.File( pro_dir +test_name + '.h5', 'w') as hf:
    hf.create_dataset(test_name,  data = test_img)

# could not broadcast input array from shape (3293,80,80,3) into shape (3293)

In [30]:
test_generator.reset()
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

Y_pred = model.predict (test_img,
                        steps=12186,
                        verbose=1)

    1/12186 [..............................] - 0s 22ms/step


In [48]:
predicted_class_indices=np.argmax(Y_pred,axis=1)
predicted_class_indices[:3]

array([0, 8, 0], dtype=int64)

In [68]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
# for i in range(0, len(predictions)): 
#     predictions[i] = int(predictions[i]) 
prediction = np.array(predictions,dtype=np.int64)

In [75]:
filenames=test_generator.filenames
results=pd.DataFrame({"filename":filenames,
                      "category":predictions})
results['category'] = results['category'].astype(np.int64)
results.to_csv(pro_dir+"results.csv",index=False)

In [76]:

results['category'][1].type
#記得把0換成00 以此類推

AttributeError: 'numpy.int64' object has no attribute 'type'

In [41]:
# for i in range(1,len(train_df)):
#     if 941%int(i)==0:
#         print(i,end=' ')

1 941